In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# import
import pandas as pd
from pprint import pprint
from zen_garden.postprocess.results import Results
from pathlib import Path
from os import chdir

# set path
path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/notebooks')
chdir(path)

In [12]:
# define import path
path_import = Path("../data/outputs/05_CCTS_WTE_small_01_cost_optimal")

# define export path
path_export = Path("/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/05_CCTS_WTE_small_02_n1")

In [13]:
# cost optimal solution
results = Results(path_import)

In [14]:
# get flow transport and save as csv
results.get_df('flow_transport', to_csv="flow_transport")

technology   edge                          time_operation
flowline     waste_1_CH-swiss_storage_CH   0                  0.00000
             waste_1_CH-waste_2_CH         0                  0.00000
             waste_1_CH-waste_9_CH         0                  0.00000
             waste_1_CH-waste_11_CH        0                  0.00000
             waste_1_CH-waste_19_CH        0                 26.08242
                                                               ...   
truck_batch  waste_19_CH-swiss_storage_CH  0                  0.00000
             waste_19_CH-waste_1_CH        0                  0.00000
             waste_19_CH-waste_2_CH        0                  0.00000
             waste_19_CH-waste_9_CH        0                  0.00000
             waste_19_CH-waste_11_CH       0                  0.00000
Name: flow_transport, Length: 75, dtype: float64

In [15]:
# get flow transport csv
df = pd.read_csv('flow_transport.csv')

In [16]:
# create nominal flow transport files

# find unique technologies
technology = df['technology'].unique()

for i in range(len(technology)):
     
    #Get table of one technology 
    tec = technology[i]
    x = df[df['technology'] == tec]
    
    #Set path
    parent_path = Path(path_export)
    path = [file for file in Path(parent_path).rglob(tec)]
    path = path[0]
    filepath = Path(str(path)+'/nominal_flow_transport.csv')
    
    #Clean up table
    x = x.drop(columns='technology')
    x = x[x['flow_transport'] != 0]
    x = x.rename(columns={'time_operation': 'year'})
    x['year'] = x['year'] + 2025 
    
    #clean up table, differentiate between node and edge
    if path.parent.name == 'set_transport_technologies':
        x = x.rename(columns={'location': 'edge'})

    else:
        x = x.rename(columns={'location': 'node'})

    #Safe
    x.to_csv(filepath, index=False)